# Issue 46: add functionality to choose minimizing or maximizing for each metric in a dataset

[See issue progress on GitHub here.](https://github.com/NREL/tyche/issues/46)

In [1]:
import os
import sys

sys.path.insert(1, os.path.abspath("../../src"))

import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

# Sustainable Aviation Fuel

In [2]:
saf_loc = "../../tutorial/data/saf"

saf_designs = ty.Designs(saf_loc)
saf_investments = ty.Investments(saf_loc)
saf_designs.compile()
tranche_results = saf_investments.evaluate_tranches(saf_designs,
                                                    sample_count=1000)

techs = tranche_results.summary.index.get_level_values('Technology').unique()

Evaluating HEFA Camelina
Evaluating HEFA Castor
Evaluating HEFA Jatropha
Evaluating HEFA Pennycress
Evaluating HEFA Yellow Grease


Evaluate the maximum metric values using the `optimum_metrics` method (formerly the `max_metrics` method). After refactoring, the method can take either a `Dict` of optimization senses, one per metric, or a `str` specifying one optimization sense to be used across all metrics. If no sense is provided, then the optimization sense used to instantiate the `EpsilonConstraintOptimizer` class object is used for all metrics.

First demonstrate the use of an optimization sense dictionary, with one sense per metric.

In [3]:
sense_dict = {'Reduction in MJSP' : 'max',
              'Reduction in Jet GHG': 'max',
              'MJSP': 'min',
              'Jet GHG': 'min'}

opt_metric_values = {}

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'min'
    )
    opt_metric_values[i] = optimizer.optimum_metrics(
        metric_sense = sense_dict,
        total_amount = 5000000
    )

opt_metric_values

{'HEFA Camelina': Jet GHG                  0.086806
 MJSP                    26.558133
 Reduction in Jet GHG     0.054194
 Reduction in MJSP        2.457708
 Name: Value, dtype: float64,
 'HEFA Castor': Jet GHG                 1.590000e-01
 MJSP                    2.864583e+01
 Reduction in Jet GHG   -9.912778e-18
 Reduction in MJSP       2.339883e+00
 Name: Value, dtype: float64,
 'HEFA Jatropha': Jet GHG                 1.650000e-01
 MJSP                    1.389508e+01
 Reduction in Jet GHG   -1.859212e-17
 Reduction in MJSP       1.131671e+00
 Name: Value, dtype: float64,
 'HEFA Pennycress': Jet GHG                 9.810000e-02
 MJSP                    2.349886e+01
 Reduction in Jet GHG   -5.859814e-18
 Reduction in MJSP       1.354039e+00
 Name: Value, dtype: float64,
 'HEFA Yellow Grease': Jet GHG                  0.041724
 MJSP                    14.452646
 Reduction in Jet GHG     0.009876
 Reduction in MJSP        1.409284
 Name: Value, dtype: float64}

Next, provide a sense (max) as a string to `optimum_metrics` that overrides the sense provided to the `EpsilonConstraintOptimizer` object.

In [4]:
max_metric_values = {}

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'min',
    )
    max_metric_values[i] = optimizer.optimum_metrics(
        metric_sense = 'max',
        total_amount = 5000000
    )

max_metric_values

{}

Optimize for minimum jet selling price (MJSP) by minimizing MJSP directly.

In [3]:
min_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'min',
    )
    w = optimizer.opt_slsqp(
        "MJSP",
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    min_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3],
        'Optimization sense' : w[5]
    }
    min_results.append(
        min_dict
    )
    del tranche_tech_results, evaluator, optimizer, min_dict, w

min_results

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Optimization terminated successfully


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            1.523678e-04
  Fatty Acid Composition        2.600000e+06
  Overall Process Efficiency    2.400000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.090830
  MJSP                    26.542196
  Reduction in Jet GHG     0.050170
  Reduction in MJSP        2.473645
  Name: Value, dtype: float64,
  'Optimization sense': 'min'},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            5.500000e+05
  Fatty Acid Composition        1.750000e+06
  Overall Process Efficiency    2.700000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.178390
  MJSP                    28.634955
  Reduction in Jet GHG    -0.019390
  Reduction in MJSP        2.350755
  Name:

Now optimize for MJSP by maximizing the reduction in MSJP from the state of technology value.

In [5]:
max_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'max',
    )
    metric_max = optimizer.max_metrics()
    w = optimizer.opt_slsqp(
        "Reduction in MJSP",
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    max_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3],
        'Maximum metrics': metric_max,
        'Optimization sense' : w[5]
    }
    max_results.append(
        max_dict
    )
    del tranche_tech_results, evaluator, optimizer, metric_max, max_dict, w

max_results

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Optimization terminated successfully


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            4.912108e-04
  Fatty Acid Composition        2.600000e+06
  Overall Process Efficiency    2.400000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.090892
  MJSP                    26.583685
  Reduction in Jet GHG     0.050108
  Reduction in MJSP        2.432156
  Name: Value, dtype: float64,
  'Maximum metrics': Jet GHG                  0.141000
  MJSP                    29.015842
  Reduction in Jet GHG     0.078079
  Reduction in MJSP        4.764545
  Name: Value, dtype: float64,
  'Optimization sense': 'max'},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            1.750016e+06
  Fatty Acid Composition        1.749999e+06
  Overall Process Efficiency    1.499985e+06
  Name: Amount, 

The metric values found from `optimizer.max_metrics()` consistently out-perform the metric values found through `opt_slsqp`.